La explicación de qué es cada término y como se obtuvo se encuentra en las notas.

In [1]:
import numpy as np
from scipy.special import erf

$$ f_1^{(AA)} = 3(2^{3/2}) \sum_{i=1}^k \sum_{j=1}^k d_i d_j \frac{(\alpha_i \alpha_j)^{7/4}}{(\alpha_i + \alpha_j)^{5/2}} $$

In [2]:
def f1AA(d, a):
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 7/4) / np.power(a[i]+a[j], 5/2)
    return 3*np.power(2, 3/2) * suma

$$ \mathrm{arg} = -\frac{a b}{a+b} |\mathbf{R}_A - \mathbf{R}_B|^2 \,, \qquad
K = \exp(\mathrm{arg})$$

In [3]:
def arg(a, b, RA, RB):
    p = a+b
    mu = a*b/p
    RAB2 = np.square(np.linalg.norm(RA-RB))
    return -mu*RAB2

In [4]:
def K(a, b, RA, RB):
    return np.exp(arg(a, b, RA, RB))

$$ f_1^{(AB)} = 2^{3/2} \sum_{i=1}^k \sum_{j=1}^k d_i d_j \frac{(\alpha_i \alpha_j)^{7/4}}{(\alpha_i + \alpha_j)^{5/2}}
\left( 3 + 2 \mathrm{arg} \right) \, K
$$

In [5]:
def f1AB(d, a, RA, RB):
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 7/4) / np.power(a[i]+a[j], 5/2) * (3 + 2*arg(a[i], a[j], RA, RB)) * K(a[i], a[j], RA, RB)
    return np.power(2, 3/2) * suma

$$ F_0(t) = \frac{1}{2} \sqrt{\frac{\pi}{t}} \mathrm{erf} (\sqrt{t}) $$

In [6]:
def F0(t):
    return (1/2) * np.sqrt(np.pi/t) * erf(np.sqrt(t))

$$ \sum_{C} Z_C F_0 \big[ (\alpha + \beta) |\mathbf{R}_P - \mathbf{R}_C|^2 \big] \,, \qquad
\mathbf{R}_P = \frac{\alpha \mathbf{R}_A + \beta \, \mathbf{R}_B}{\alpha + \beta}
$$

In [7]:
def ZC(a, b, RA, RB, Z, R):
    """
        Suma sobre núcleos
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    R : vector de coords. de centros [RA, RB, ..., RC] donde RA = [a1, a2, a3]

    len(R) = len(Z)
    """
    if np.array_equal(RA,RB): # RA = RB
        RP = RA
    else: # RA != RB
        RP = (a*RA+b*RB)/(a+b)
    
    suma = 0
    for C in range(len(Z)):
        RPC2 = np.square(np.linalg.norm(RP-R[C]))
        suma += Z[C] * F0((a+b)*RPC2)
    return suma

$$ f_2^{(AA)} = -\frac{2^{5/2}}{\pi^{1/2}} Z_A \sum_{i=1}^k \sum_{j=1}^k d_i d_j \frac{(\alpha_i \alpha_j)^{3/4}}{\alpha_i + \beta_j} + f_2^{(AB)} $$

Para el término $f_2^{(AB)}$ hay que considerar que $\mathbf{R}_A = \mathbf{R}_B$ y nunca ocurre que $\mathbf{R}_C = \mathbf{R}_A$.

In [8]:
def f2AA(d, a, RA, RB, Z):
    """
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    """
    k = len(d)

    ZA = Z[0]
    suma1 = 0
    for i in range(k):
        for j in range(k):
            suma1 = d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j])
    suma1 = ZA*suma1

    ZB = Z[1]
    suma2 = 0
    for i in range(k):
        for j in range(k):
            suma2 = d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j]) * ZC(a[i], a[j], RA, RA, [ZB], [RB])
    
    return -np.power(2, 5/2) / np.sqrt(np.pi) * (suma1 + suma2)

$$ f_2^{(AB)} = -\frac{2^{5/2}}{\pi^{1/2}} \sum_{i=1}^k \sum_{j=1}^k d_i d_j \frac{(\alpha_i \alpha_j)^{3/4}}{\alpha_i + \beta_j} \, K
\sum_C Z_C F_0 \big[ (\alpha + \beta) |\mathbf{R}_P - \mathbf{R}_C|^2 \big]
$$


In [9]:
def f2AB(d, a, RA, RB, Z):
    """
        Suma sobre núcleos
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    """
    R = [RA, RB]
    RAB2 = np.square(np.linalg.norm(RA-RB))
    
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j]) * K(a[i], a[j], RA, RB) * ZC(a[i], a[j], RA, RB, Z, R)
    return -np.power(2, 5/2) / np.sqrt(np.pi)

$$ f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

In [10]:
def fpp(p, d, a, RA, RB, Z):
    """
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    """
    t1AA = f1AA(d,a)
    t1AB = f1AB(d, a, RA, RB)
    t2AA = f2AA(d, a, RA, RB, Z)
    t2AB = f2AB(d, a, RA, RB, Z)

    if p in [1,2]:
        return 2*(t1AA + t1AB + t2AA + t2AB)
    else:
        return 2*(t1AA - t1AB + t2AA - t2AB)

In [11]:
# Parámetros Gaussianos obtenidos en `(1) STO-3G.ipynb` para zeta=1.24
d = np.array([0.44471812476789035, 0.5352716544572346, 0.1543000507808527])
a = np.array([0.16887939463273338, 0.6240343336327064, 3.4256944279866635])

RA = np.array([0, 0, 0])
RB = np.array([1.4, 0, 0]) # distancia interatómica de 1.4
ZA, ZB = 1, 1 # carga nuclear del hidrógeno
Z = [ZA, ZB]

f11 = fpp(1, d, a, RA, RB, Z)
f33 = fpp(3, d, a, RA, RB, Z)

print('f11 & f22:', f11)
print('f33 & f44:', f33)

f11 & f22: -4.564734200242766
f33 & f44: 7.255605621340926


In [12]:
# revisión: fAA
print('f1AA:', f1AA(d, a))

f1AA: 0.7600435455041736


In [13]:
# revisión: f1AB = f1BA
print('f1AB:', f1AB(d, a, RA, RB))
print('f1BA:', f1AB(d, a, RB, RA))

f1AB: 0.2364532878155391
f1BA: 0.2364532878155391


In [14]:
# revisión: f2AA = f2BB
print('f2AA:', f2AA(d, a, RA, RB, Z))
print('f2BB:', f2AA(d, a, RB, RA, Z))

f2AA: -0.08732569022963345
f2BB: -0.08732569022963345


In [15]:
# revisión: f2AA = f2BB
print('f2AB:', f2AB(d, a, RA, RB, Z))
print('f2BA:', f2AB(d, a, RB, RA, Z))

f2AB: -3.191538243211462
f2BA: -3.191538243211462
